# Step 3: Kafka Producer Implementation

**Objective:**  
Stream telemetry data from cache at the real telemetry frequency.

**Instructions:**
- Use kafka-python package
- For each cached row, send JSON payload to Kafka topic ("f1-speed-stream") every X ms (match source data frequency)
- Validate: Message arrives in topic, order preserved, frequency correct


In [ ]:
# Import required libraries
import sys
import os
import time

# Add project root to Python path
# In Jupyter, getcwd() typically returns the project root
# If not, navigate up from notebooks/ directory
current_dir = os.getcwd()
if os.path.basename(current_dir) == 'notebooks':
    # We're in notebooks/ directory, go up one level
    project_root = os.path.dirname(current_dir)
else:
    # We're already at project root
    project_root = current_dir

# Add to path if not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.kafka_producer import RealTimeProducer
from src.cache_manager import CacheManager


✅ Imports successful


In [ ]:
# Initialize producer
producer = RealTimeProducer(config_path="../config/config.yaml")


✅ Producer initialized


In [ ]:
# Load data into cache
data_dir = "../data"
data_files = [f for f in os.listdir(data_dir) if f.endswith(('.csv', '.parquet'))]

if data_files:
    latest_file = max(data_files, key=lambda f: os.path.getctime(os.path.join(data_dir, f)))
    file_path = os.path.join(data_dir, latest_file)
    format_type = 'parquet' if latest_file.endswith('.parquet') else 'csv'
    
    producer.load_cache(file_path, format=format_type)


INFO:src.cache_manager:Loading data from ../data/2023_Monaco_Race_20251119_182219.parquet (parquet format)...
INFO:src.cache_manager:Loaded 982330 records
INFO:src.utils:Calculated frequency: 999848759.83 Hz (avg interval: 0.00 ms)
INFO:src.cache_manager:Cache loaded successfully. Frequency: 999848759.83 Hz


✅ Cache loaded: 982330 records
✅ Frequency: 999848759.83 Hz


In [ ]:
# ============================================================================
# VERIFICATION MODE: Test Producer (Short Duration)
# ============================================================================
# This section verifies the producer works correctly with a short test stream
# For continuous real-time streaming, see cells below or use the consumer notebook
# ============================================================================

import socket
from kafka import KafkaProducer

kafka_config = producer.config.get('kafka', {})
bootstrap_servers = kafka_config.get('bootstrap_servers', 'localhost:9092')
host, port = bootstrap_servers.split(':')
port = int(port)

try:
    # Try to connect to Kafka
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.settimeout(2)
    result = sock.connect_ex((host, port))
    sock.close()
    
    if result == 0:
        print(f"✅ Kafka is running on {bootstrap_servers}")
    else:
        raise ConnectionError(f"Cannot connect to Kafka on {bootstrap_servers}")
except Exception as e:
    print(f"❌ Kafka is not running or not accessible: {e}")
    print("\n📋 To start Kafka:")
    print("   1. Install Kafka:")
    print("      - Option A (Homebrew): brew install kafka")
    print("      - Option B (Manual): Download from https://kafka.apache.org/downloads")
    print("   2. Start Kafka:")
    print("      bash scripts/start_kafka.sh")
    print("   3. Or manually:")
    print("      # Start ZooKeeper")
    print("      $KAFKA_HOME/bin/zookeeper-server-start.sh $KAFKA_HOME/config/zookeeper.properties &")
    print("      # Start Kafka (wait 5 seconds after ZooKeeper)")
    print("      $KAFKA_HOME/bin/kafka-server-start.sh $KAFKA_HOME/config/server.properties &")
    raise

# Start streaming to Kafka (test with short duration)
print("\n🚀 Starting TEST stream (30 seconds)...")
print("   This is a verification test. For continuous streaming, see cells below.\n")

# Stream for 30 seconds as a test
stats = producer.stream_at_frequency(duration_seconds=30, speed_factor=10.0)  # 10x speed for faster testing

print("\n" + "=" * 70)
print("📊 VERIFICATION TEST RESULTS:")
print("=" * 70)
print(f"  ✅ Messages sent: {stats.get('messages_sent', 0):,}")
print(f"  ⏱️  Duration: {stats.get('duration_seconds', 0):.2f} seconds")
print(f"  📈 Avg rate: {stats.get('avg_rate_messages_per_sec', 0):.2f} msg/s")
print(f"  🔄 Frequency: {stats.get('frequency_hz', 0):.2f} Hz")
print(f"  ✅ Success: {stats.get('success', False)}")
print("=" * 70)
print("\n💡 Producer verification complete!")
print("   For continuous streaming (needed for consumer/analysis), see cells below.")


INFO:kafka.conn:<BrokerConnection client_id=kafka-python-producer-1, node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]


VERIFICATION MODE: Testing Producer with Short Stream

Checking Kafka availability...
✅ Kafka is running on localhost:9092

🚀 Starting TEST stream (30 seconds)...
   This is a verification test. For continuous streaming, see cells below.



INFO:kafka.conn:<BrokerConnection client_id=kafka-python-producer-1, node_id=bootstrap-0 host=localhost:9092 <checking_api_versions_recv> [IPv6 ('::1', 9092, 0, 0)]>: Broker version identified as 2.6
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-producer-1, node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
INFO:src.kafka_producer:Connected to Kafka at localhost:9092
INFO:src.kafka_producer:Starting stream to topic 'f1-speed-stream' at 999848759.83 Hz (speed: 10.0x)
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-producer-1, node_id=1 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-producer-1, node_id=1 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-producer-1, node_id=bootstrap-0 host=localhost:9092 


📊 VERIFICATION TEST RESULTS:
  ✅ Messages sent: 221,270
  ⏱️  Duration: 30.00 seconds
  📈 Avg rate: 7374.88 msg/s
  🔄 Frequency: 999848759.83 Hz
  ✅ Success: True

💡 Producer verification complete!
   For continuous streaming (needed for consumer/analysis), see cells below.


In [5]:
# ============================================================================
# VALIDATION: Verify Messages Arrived in Topic
# ============================================================================
print("\n" + "=" * 70)
print("VALIDATION: Checking if messages arrived in topic")
print("=" * 70)
print("\nValidating message delivery...")

from kafka import KafkaConsumer
from kafka.errors import KafkaError
import json

kafka_config = producer.config.get('kafka', {})
bootstrap_servers = kafka_config.get('bootstrap_servers', 'localhost:9092')
topic_name = kafka_config.get('topic_name', 'f1-speed-stream')

try:
    # Try to consume messages to verify delivery and check topic exists
    print("Consuming sample messages to verify delivery...")
    consumer = KafkaConsumer(
        topic_name,
        bootstrap_servers=bootstrap_servers,
        auto_offset_reset='earliest',
        consumer_timeout_ms=5000,  # 5 second timeout
        value_deserializer=lambda m: json.loads(m.decode('utf-8')) if m else None
    )
    
    # Check topic metadata from consumer (this will fail if topic doesn't exist)
    try:
        partitions = consumer.partitions_for_topic(topic_name)
        if partitions:
            print(f"✅ Topic '{topic_name}' exists with {len(partitions)} partition(s)")
        else:
            print(f"⚠️ Warning: Topic '{topic_name}' found but no partitions")
    except Exception as e:
        print(f"⚠️ Could not get topic metadata: {e}")
    
    messages_received = []
    message_count = 0
    max_sample = 5  # Sample first 5 messages
    
    for message in consumer:
        messages_received.append(message.value)
        message_count += 1
        if message_count >= max_sample:
            break
    
    consumer.close()
    
    if message_count > 0:
        print(f"✅ Successfully received {message_count} message(s) from topic")
        print(f"\nSample message (first one):")
        sample = messages_received[0]
        print(f"  Driver: {sample.get('driver', 'N/A')}")
        print(f"  Speed: {sample.get('speed', 'N/A')} km/h")
        print(f"  SessionTime: {sample.get('session_time', 'N/A')}")
        print(f"  Timestamp: {sample.get('timestamp', 'N/A')}")
        
        print(f"\n✅ Message delivery validated successfully!")
        print(f"   Total messages in topic: {message_count}+ (sampled first {max_sample})")
    else:
        print("⚠️ No messages found in topic")
        print("   This could mean:")
        print("   - Messages were sent but already consumed")
        print("   - Producer didn't send messages successfully")
        print("   - Topic is empty")
        print("\n   To manually verify, run:")
        print(f"   kafka-console-consumer.sh --bootstrap-server {bootstrap_servers} --topic {topic_name} --from-beginning")
    
except KafkaError as e:
    print(f"❌ Error validating messages: {e}")
    print("\n   To manually verify, run:")
    print(f"   kafka-console-consumer.sh --bootstrap-server {bootstrap_servers} --topic {topic_name} --from-beginning")
except Exception as e:
    print(f"❌ Error during validation: {e}")
    print("\n   To manually verify, run:")
    print(f"   kafka-console-consumer.sh --bootstrap-server {bootstrap_servers} --topic {topic_name} --from-beginning")

# Close producer
producer.close()
print("\n✅ Step 3 Complete: Kafka producer streaming successful!")


INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=bootstrap-0 host=localhost:9092 <connecting> [IPv6 ('::1', 9092, 0, 0)]>: connecting to localhost:9092 [('::1', 9092, 0, 0) IPv6]
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=bootstrap-0 host=localhost:9092 <checking_api_versions_recv> [IPv6 ('::1', 9092, 0, 0)]>: Broker version identified as 2.6
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-2.2.15, node_id=bootstrap-0 host=localhost:9092 <connected> [IPv6 ('::1', 9092, 0, 0)]>: Connection complete.
INFO:kafka.consumer.subscription_state:Updating subscribed topics to: ('f1-speed-stream',)
INFO:kafka.consumer.subscription_state:Updated partition assignment: [TopicPartition(topic='f1-speed-stream', partition=0), TopicPartition(topic='f1-speed-stream', partition=1), TopicPartition(topic='f1-speed-stream', partition=2), TopicPartition(topic='f1-speed-stream', partition=3)]
INFO:kafka.conn:<BrokerConnection client_id=kafka-python-


VALIDATION: Checking if messages arrived in topic

Validating message delivery...
Consuming sample messages to verify delivery...
✅ Topic 'f1-speed-stream' exists with 4 partition(s)
✅ Successfully received 5 message(s) from topic

Sample message (first one):
  Driver: SAR
  Speed: 0.0 km/h
  SessionTime: 0.0
  Timestamp: 2025-11-16T15:27:03.867770

✅ Message delivery validated successfully!
   Total messages in topic: 5+ (sampled first 5)

✅ Step 3 Complete: Kafka producer streaming successful!


# ============================================================================
# CONTINUOUS STREAMING MODE (For Consumer/Analysis Testing)
# ============================================================================

**When to use this:**
- When running the consumer notebook (`05_kafka_consumer.ipynb`)
- When running ML model inference (`06_ml_model_training.ipynb`)
- When running live dashboard (`08_live_dashboard.ipynb`)

**How it works:**
- The producer runs continuously in the background
- Streams data at configurable speed (default: 10x faster for testing)
- Can loop data continuously for extended testing
- Automatically starts when you run the consumer/analysis notebooks

**Options:**
1. **Auto-start from consumer** (Recommended) - Producer starts automatically when consumer runs
2. **Manual start in this notebook** (Cell 7) - Start producer manually here
3. **Background script** (Cell 8) - Use standalone script for terminal execution


In [6]:
# Option 1: Start Producer in Background (Recommended for Consumer Testing)
# This starts the producer as a background process that runs continuously
# Perfect for running consumer/analysis notebooks in parallel

from src.producer_manager import ensure_producer_running, is_producer_running, stop_producer

print("=" * 70)
print("OPTION 1: Start Producer in Background")
print("=" * 70)
print("\nThis will start the producer as a background process.")
print("You can then run the consumer/analysis notebooks while producer streams data.\n")

# Check if already running
if is_producer_running():
    print("ℹ️  Producer is already running in background")
    print("   You can proceed to run the consumer notebook now!")
else:
    # Start producer in background
    # speed_factor=10.0 means 10x faster than real-time (good for testing)
    # Set loop=True to continuously replay data
    success = ensure_producer_running(
        speed_factor=10.0,  # 10x speed for faster testing (change to 1.0 for real-time)
        loop=True,          # Loop data continuously (replay from beginning when finished)
        project_root=project_root
    )
    
    if success:
        print("\n✅ Producer started successfully!")
        print("   You can now:")
        print("   1. Run the consumer notebook (05_kafka_consumer.ipynb)")
        print("   2. Run ML model inference (06_ml_model_training.ipynb)")
        print("   3. Run live dashboard (08_live_dashboard.ipynb)")
        print("\n   The producer will stream data continuously in the background.")
        print("   To stop: Run the cell below or use: stop_producer()")
    else:
        print("\n❌ Failed to start producer. Check logs: producer.log")


OPTION 1: Start Producer in Background

This will start the producer as a background process.
You can then run the consumer/analysis notebooks while producer streams data.

ℹ️  Producer is already running in background
   You can proceed to run the consumer notebook now!


# Option 2: Stop Producer (if running)
print("=" * 70)
print("OPTION 2: Stop Background Producer")
print("=" * 70)

if is_producer_running():
    response = input("\n⚠️  Producer is running. Stop it? (y/n): ")
    if response.lower() == 'y':
        stop_producer()
        print("✅ Producer stopped")
    else:
        print("ℹ️  Producer continues running")
else:
    print("ℹ️  No producer process is currently running")


## Option 3: Manual Background Script (Terminal)

For running the producer from terminal (outside notebook):

```bash
# Navigate to project directory
cd "/Users/Ujjwal/Documents/DE Project/f1_streaming_pipeline"

# Run in background with 10x speed and looping
nohup python scripts/run_producer.py --speed-factor 10.0 --loop > producer.log 2>&1 &

# Check if running
ps aux | grep run_producer

# View logs
tail -f producer.log

# Stop the producer
pkill -f run_producer.py
```
